In [ ]:
# pip install transformers==4.28.1
# pip install chardet==5.1.0
# pip install sentencepiece==0.1.99
# pip install protobuf==3.20

mBART-50 模型需要大約 15GB 的 GPU 記憶體

In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers.optimization import AdamW
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Arabic (ar_AR), Czech (cs_CZ), German (de_DE), English (en_XX), Spanish (es_XX), Estonian (et_EE), Finnish (fi_FI), French (fr_XX), Gujarati (gu_IN), Hindi (hi_IN), Italian (it_IT), Japanese (ja_XX), Kazakh (kk_KZ), Korean (ko_KR), Lithuanian (lt_LT), Latvian (lv_LV), Burmese (my_MM), Nepali (ne_NP), Dutch (nl_XX), Romanian (ro_RO), Russian (ru_RU), Sinhala (si_LK), Turkish (tr_TR), Vietnamese (vi_VN), Chinese (zh_CN), Afrikaans (af_ZA), Azerbaijani (az_AZ), Bengali (bn_IN), Persian (fa_IR), Hebrew (he_IL), Croatian (hr_HR), Indonesian (id_ID), Georgian (ka_GE), Khmer (km_KH), Macedonian (mk_MK), Malayalam (ml_IN), Mongolian (mn_MN), Marathi (mr_IN), Polish (pl_PL), Pashto (ps_AF), Portuguese (pt_XX), Swedish (sv_SE), Swahili (sw_KE), Tamil (ta_IN), Telugu (te_IN), Thai (th_TH), Tagalog (tl_XX), Ukrainian (uk_UA), Urdu (ur_PK), Xhosa (xh_ZA), Galician (gl_ES), Slovene (sl_SI)

In [22]:
MODEL_ID = "../models/facebook_mbart-large-50"
src_lang = "en_XX"
tgt_lang = "zh_CN" #ro_RO

tokenizer = MBart50TokenizerFast.from_pretrained(MODEL_ID, src_lang=src_lang, tgt_lang=tgt_lang)

In [3]:
model = MBartForConditionalGeneration.from_pretrained(MODEL_ID)

In [23]:
# Train
src_text = " UN Chief Says There Is No Military Solution in Syria"
tgt_text =  "Şeful ONU declară că nu există o soluţie militară în Siria"

src_text = "How are you?"
tgt_text =  "你好嗎?"

model_inputs = tokenizer(src_text, return_tensors="pt")
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_text, return_tensors="pt").input_ids

# Set up the optimizer and training settings
optimizer = AdamW(model.parameters(), lr=1e-4)
model.train()

/home/flash/miniconda3/envs/whisper/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/flash/miniconda3/envs/whisper/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [24]:
print('Fine-tuning started')
for i in range(10):
    optimizer.zero_grad()
    output = model(**model_inputs, labels=labels) # forward pass
    loss = output.loss
    loss.backward()
    optimizer.step()
print('Fine-tuning ended')

Fine-tuning started
Fine-tuning ended


In [32]:
# translate French to English
src_text = "yo?"
# tokenizer = MBart50TokenizerFast.from_pretrained(MODEL_ID)
# tokenizer.src_lang = src_lang
# tokenizer = MBart50TokenizerFast.from_pretrained(MODEL_ID, src_lang=src_lang, tgt_lang=tgt_lang)
encoded_src = tokenizer(src_text, return_tensors="pt")
print(f"{encoded_src=}")
generated_tokens = model.generate(**encoded_src, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(translation)

encoded_src={'input_ids': tensor([[250004,   3005,     32,      2]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
['你好嗎?']
